In [1]:
import numpy as np
import pandas as pd 
import sympy as sp
import scipy.integrate
import altair as alt 
import diaux.viz
colors, palette = diaux.viz.altair_style()


In [2]:
nu_max = sp.Symbol(r'{{\nu^{(max)}}}')
gamma_max = sp.Symbol(r'{{\gamma^{(max)}}}')
theta_a = sp.Symbol(r'{{\theta_a}}')
theta_0 = sp.Symbol(r'{{\theta_0}}')
K_M = sp.Symbol(r'{{K_M}}')
c = sp.Symbol('c')
beta = sp.Symbol(r'\beta')
phi_R = sp.Symbol('{{\phi_R}}')
phi_P = sp.Symbol('{{\phi_P}}')
nu_exp = nu_max * (1 + K_M / c)**-1
gamma_exp = gamma_max * (1 + theta_0/theta_a)**-1
eq = theta_a - beta * ((nu_exp/gamma_exp) * (phi_P/phi_R) - 1)
soln = sp.solve(eq, theta_a)



In [ ]:
c_x = sp.Symbol('c_x')
km_x = sp.Symbol('{{K_{M,x}}}')
phiy_steady = sp.Symbol('{{phi_y}}')

In [9]:
soln[0]

(-\beta*(c*{{\gamma^{(max)}}}*{{\phi_R}} - c*{{\nu^{(max)}}}*{{\phi_P}} + {{K_M}}*{{\gamma^{(max)}}}*{{\phi_R}}) + sqrt(\beta*(\beta*c**2*{{\gamma^{(max)}}}**2*{{\phi_R}}**2 - 2*\beta*c**2*{{\gamma^{(max)}}}*{{\nu^{(max)}}}*{{\phi_P}}*{{\phi_R}} + \beta*c**2*{{\nu^{(max)}}}**2*{{\phi_P}}**2 + 2*\beta*c*{{K_M}}*{{\gamma^{(max)}}}**2*{{\phi_R}}**2 - 2*\beta*c*{{K_M}}*{{\gamma^{(max)}}}*{{\nu^{(max)}}}*{{\phi_P}}*{{\phi_R}} + \beta*{{K_M}}**2*{{\gamma^{(max)}}}**2*{{\phi_R}}**2 + 4*c**2*{{\gamma^{(max)}}}*{{\nu^{(max)}}}*{{\phi_P}}*{{\phi_R}}*{{\theta_0}} + 4*c*{{K_M}}*{{\gamma^{(max)}}}*{{\nu^{(max)}}}*{{\phi_P}}*{{\phi_R}}*{{\theta_0}})))/(2*{{\gamma^{(max)}}}*{{\phi_R}}*(c + {{K_M}}))

In [11]:
def single_nutrient(params, t0, gamma_max, nu_max, phi_R, phi_P, theta_0, Km, Y):
    """
    Defines the complete dynamical model for growth on a single carbon source. 

    Parameters
    ----------
    params: list, of order [M, theta_a, c, M_P]
        List of the model parameters which
    """

    M, theta_a, c, M_P = params

    # Define the expressions for the translational and nutritional capacities
    gamma = gamma_max * theta_a / (theta_a + theta_0)
    nu = nu_max * c / (c + Km)

    # Equation 1: Proteome Mass Dynamics
    dM_dt = phi_R * M * gamma

    # Metabolic protein synthesis
    dMp_dt = phi_P  * dM_dt 

    # Nutrient influx
    dtheta_dt = nu * M_P

    # Nutrient consumption
    dc_dt = - dtheta_dt / Y 

    return [dM_dt, dtheta_dt - dM_dt, dc_dt, dMp_dt]




In [17]:
gamma_max = (17.1 * 3600 / 7459)
nu_max = 2.4
theta_0 = 0.0013 * 20 # in M
Km = 0.005 # in mM
Y = 0.377

phi_R = 0.35
phi_P = 0.45

# Define the intial conditions
M = 0.001
theta_a = 0.0001
M_P = phi_P * M 
c = 0.010

# Integrate
n_timesteps = 500
t_stop = 20
delta_t = t_stop/n_timesteps
t = np.linspace(0, t_stop, n_timesteps)
out = scipy.integrate.odeint(single_nutrient, [M, theta_a, c, M_P], t, args=(gamma_max,nu_max, phi_R, phi_P, theta_0, Km, Y))


In [18]:
out

array([[1.00000000e-03, 1.00000000e-04, 1.00000000e-02, 4.50000000e-04],
       [1.00050619e-03, 1.28322009e-04, 9.92353264e-03, 4.50227784e-04],
       [1.00113726e-03, 1.56461157e-04, 9.84721905e-03, 4.50511768e-04],
       ...,
       [4.86758604e-03, 2.41389996e-06, 1.57779828e-10, 2.19041372e-03],
       [4.86763767e-03, 2.36226408e-06, 1.80062392e-10, 2.19043695e-03],
       [4.86768820e-03, 2.31172508e-06, 2.03952455e-10, 2.19045969e-03]])

In [24]:
_df = pd.DataFrame(out, columns=['dM_dt', 'dtheta_dt', 'dc_dt', 'dMp_dt'])
_df['rel_M'] = _df['dM_dt'] / M
_df['time'] = t
alt.Chart(_df).mark_line().encode(x='time:Q', y='rel_M:Q')

alt.Chart(...)